# Carbon Emissions

Carbon emissions prices directly influence energy market stocks by increasing operating costs for high-emission firms (coal/gas), reducing their profitability and stock prices, while simultaneously enhancing the competitiveness and valuation of renewable energy firms. Higher carbon prices, particularly within systems like the EU ETS, force fossil fuel companies to pay for emissions, which acts as a penalty that reduces their market value. 

#### Source: U.S. Energy Information Administration (EIA) – Monthly Energy Review (Table 11.1) 
- This dataset was obtained from the Monthly Energy Review (MER) published by the U.S.  
- Energy Information Administration. Specifically, Table 11.1: “Carbon Dioxide Emissions from Energy Consumption by Source” reports monthly U.S. CO₂ emissions from fossil fuel energy use, disaggregated by major fuel types. 
- Publisher: U.S. Energy Information Administration (EIA) 
- Access Method: Manual download of Excel file (Table 11.01.xlsx) from EIA Monthly Energy Review

In [37]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------

In [39]:
import pandas as pd
from pathlib import Path

# === Paths ===
FILE = r"D:/MS_Data_Science_Thesis/Data_Extraction/Downloaded_datasets/Table 11.01.xlsx"
OUTDIR = Path(r"D:/MS_Data_Science_Thesis/Data_Extraction/Raw_Data_Folder")
OUTDIR.mkdir(parents=True, exist_ok=True)

# === 1. Read sheet & skip banners ===
df = pd.read_excel(FILE, sheet_name="Monthly Data", header=8)

# keep only the first 5 meaningful columns (month + 4 fuels)
df = df.iloc[:, :5].copy()
df.columns = ["month", "coal", "natural_gas", "aviation_gasoline", "distillate_fuel_oil"]

# === 2. Parse "month" column into datetime ===
# It looks like "1973 January" -> convert to 1973-01-01
df["month"] = (pd.to_datetime(df["month"].astype(str).str.strip(), format="%Y %B", errors="coerce"))

# drop rows without a valid date
df = df.dropna(subset=["month"]).reset_index(drop=True)

# === 3. Convert numeric values ===
for c in ["coal", "natural_gas", "aviation_gasoline", "distillate_fuel_oil"]:
    df[c] = (
        df[c].astype(str)
        .str.replace(",", "")
        .str.replace("\u2212", "-")
        .str.extract(r"([-+]?\d*\.?\d+)")[0]
        .astype(float) )

# === 4. Compute total CO₂ (MMT) ===
df["total"] = df[["coal", "natural_gas", "aviation_gasoline", "distillate_fuel_oil"]].sum(axis=1)

# === 5. Save outputs ===
byfuel_out = OUTDIR / "us_co2_emissions_monthly_byfuel.csv"
total_out  = OUTDIR / "us_co2_emissions_monthly_total.csv"

df[["month", "coal", "natural_gas", "aviation_gasoline", "distillate_fuel_oil", "total"]].to_csv(byfuel_out, index=False)
df[["month", "total"]].rename(columns={"month": "date", "total": "co2_mmt"}).to_csv(total_out, index=False)

print("✅ Files saved:")
print(" -", byfuel_out)
print(" -", total_out)

display(df.head(10))


✅ Files saved:
 - D:\MS_Data_Science_Thesis\Data_Extraction\Raw_Data_Folder\us_co2_emissions_monthly_byfuel.csv
 - D:\MS_Data_Science_Thesis\Data_Extraction\Raw_Data_Folder\us_co2_emissions_monthly_total.csv


,month,coal,natural_gas,aviation_gasoline,distillate_fuel_oil,total
0,1973-01-01,109.559,125.599,0.428,55.157,290.743
1,1973-02-01,98.839,113.788,0.431,51.771,264.829
2,1973-03-01,98.488,107.736,0.459,44.460,251.143
3,1973-04-01,94.155,97.871,0.446,33.973,226.445
4,1973-05-01,95.429,92.070,0.549,35.591,223.639
5,1973-06-01,98.894,81.684,0.471,31.175,212.224
6,1973-07-01,104.377,82.894,0.420,31.004,218.695
7,1973-08-01,106.637,84.126,0.668,34.029,225.460
8,1973-09-01,99.442,81.143,0.462,34.466,215.513
9,1973-10-01,102.230,90.692,0.550,39.032,232.504
